In [1]:
import glob
import os
import librosa
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.contrib.rnn.python.ops import rnn, rnn_cell
import numpy as np
from python_speech_features import mfcc
%matplotlib inline
plt.style.use('ggplot')

In [2]:
def windows(data, window_size):
    start = 0
    while start < len(data):
        yield start, start + window_size
        start += (window_size / 2)

def extract_features(parent_dir,sub_dirs,file_ext="*.wav",bands = 13, frames = 25):
    window_size = 512 * (frames - 1)
    mfccs = []
    labels = []
    for l, sub_dir in enumerate(sub_dirs):
        for fn in glob.glob(os.path.join(parent_dir, sub_dir, file_ext)):
            #print fn
            sound_clip,s = librosa.load(fn)
            #print fn
            #print fn.split('/')[5].split('-')[0]
            label = fn.split('/')[5].split('-')[0]
            if(label=='n'):
                label=0
            if(label=='sp'):
                label=1
            for (start,end) in windows(sound_clip,window_size):
                if(len(sound_clip[start:end]) == window_size):
                    signal = sound_clip[start:end]
                    mfcc = librosa.feature.mfcc(y=signal, sr=s, n_mfcc = bands).T.flatten()[:, np.newaxis].T
                    mfccs.append(mfcc)
                    for ho in range(0,frames):
                        labels.append(label)         
    features = np.asarray(mfccs).reshape(len(mfccs)*frames,bands)
    return np.array(features), np.array(labels,dtype = np.int)

def one_hot_encode(labels):
    n_labels = len(labels)
    n_unique_labels = len(np.unique(labels))
    one_hot_encode = np.zeros((n_labels,n_unique_labels))
    one_hot_encode[np.arange(n_labels), labels] = 1
    return one_hot_encode

In [55]:


parent_dir = '/home/samim/audiotrainingset/'

tr_sub_dirs = ['/home/samim/audiotrainingset/training/']
tr_features,tr_labels = extract_features(parent_dir,tr_sub_dirs)
tr_labels = one_hot_encode(tr_labels)






In [64]:
ts_sub_dirs = ['/home/samim/audiotrainingset/testing/']
ts_features,ts_labels = extract_features(parent_dir,ts_sub_dirs)
ts_labels = one_hot_encode(ts_labels)
ts_labels

array([[ 0.,  1.],
       [ 0.,  1.],
       [ 0.,  1.],
       [ 0.,  1.],
       [ 0.,  1.],
       [ 0.,  1.],
       [ 0.,  1.],
       [ 0.,  1.],
       [ 0.,  1.],
       [ 0.,  1.],
       [ 0.,  1.],
       [ 0.,  1.],
       [ 0.,  1.],
       [ 0.,  1.],
       [ 0.,  1.],
       [ 0.,  1.],
       [ 0.,  1.],
       [ 0.,  1.],
       [ 0.,  1.],
       [ 0.,  1.],
       [ 0.,  1.],
       [ 0.,  1.],
       [ 0.,  1.],
       [ 0.,  1.],
       [ 0.,  1.],
       [ 0.,  1.],
       [ 0.,  1.],
       [ 0.,  1.],
       [ 0.,  1.],
       [ 0.,  1.],
       [ 0.,  1.],
       [ 0.,  1.],
       [ 0.,  1.],
       [ 0.,  1.],
       [ 0.,  1.],
       [ 0.,  1.],
       [ 0.,  1.],
       [ 0.,  1.],
       [ 0.,  1.],
       [ 0.,  1.],
       [ 0.,  1.],
       [ 0.,  1.],
       [ 0.,  1.],
       [ 0.,  1.],
       [ 0.,  1.],
       [ 0.,  1.],
       [ 0.,  1.],
       [ 0.,  1.],
       [ 0.,  1.],
       [ 0.,  1.],
       [ 1.,  0.],
       [ 1.,  0.],
       [ 1.,

In [62]:
np.length(ts_labels)

AttributeError: 'module' object has no attribute 'length'

In [4]:
#for a singledata/audio sample
parent_dir = '/home/samim/audiotrainingset/'
single_dirs = ['/home/samim/audiotrainingset/mydata/']
single_features,single_labels = extract_features(parent_dir,single_dirs)
single_labels = one_hot_encode(single_labels)

In [48]:
single_labels

array([[ 1.,  0.],
       [ 1.,  0.],
       [ 1.,  0.],
       ..., 
       [ 0.,  1.],
       [ 0.,  1.],
       [ 0.,  1.]])

In [5]:
# Save the mfcc and labels into a pickle file.
import cPickle as pickle
pickle.dump( tr_features, open( "tr_features.p", "wb" ) )
pickle.dump( tr_labels , open( "tr_labels.p", "wb" ) )
pickle.dump( ts_features, open( "ts_features.p", "wb" ) )
pickle.dump( ts_labels , open( "ts_labels.p", "wb" ) )

NameError: name 'tr_features' is not defined

In [6]:
# Load the  mfcc and labels back from the pickle file.
import cPickle as pickle
tr_features = pickle.load( open( "tr_features.p", "rb" ) )
tr_labels = pickle.load( open( "tr_labels.p", "rb" ) )
ts_features = pickle.load( open( "ts_features.p", "rb" ) )
ts_labels = pickle.load( open( "ts_labels.p", "rb" ) )

In [7]:
print np.shape(tr_features)
print np.shape(ts_features)
print np.shape(tr_labels)
print np.shape(ts_labels)
print np.shape(single_features)
print np.shape(single_labels)


(40450, 13)
(2700, 13)
(40450, 2)
(2700, 2)
(10700, 13)
(10700, 2)


In [8]:
print tr_features
#print ts_features
print single_features


[[  28.15841643  135.38226361  -57.07620374 ...,    1.56619996
    -7.09202049  -31.50583012]
 [  24.09893125  137.90576327 -114.65619221 ...,    2.76746728
   -13.80398321  -40.81468196]
 [ -88.58701285   93.49443242 -217.68395476 ...,   -7.49118347
     4.11981853  -29.8041856 ]
 ..., 
 [-365.26763342  221.46193591  -75.21476963 ...,  -32.07783477
    -9.96381627  -12.28702448]
 [-301.75540683  183.0349974   -67.3393457  ...,  -32.55723463
   -17.17785367   -1.30824554]
 [-277.44060537  159.09760949  -56.63679668 ...,  -33.11707145
   -18.06695419    3.47183371]]
[[ -1.63014319e+02   1.19716560e+02  -3.01664719e+01 ...,  -1.20770038e+00
   -1.23764010e+01   8.25812088e+00]
 [ -2.31086079e+02   1.19702416e+02  -3.01851377e+01 ...,  -1.23331644e+00
   -1.23827580e+01   8.28874059e+00]
 [ -8.50596566e+02  -2.72998029e+00  -2.16464621e+00 ...,  -3.93243938e+00
   -2.87728434e-01   4.15265579e+00]
 ..., 
 [ -3.56537802e+02   1.85095644e+02   1.66250130e+01 ...,  -2.10503693e+01
   -3.3575

In [46]:
ts_features

array([[-255.45413705,  198.71187532,  -79.96107757, ...,   -1.44079077,
          10.4169873 ,  -16.62076688],
       [-184.36090782,  216.22932919, -113.54443459, ...,  -12.52371133,
          14.98350096,  -22.34927891],
       [-151.1976834 ,  225.74471473, -124.068053  , ...,  -17.00080968,
          16.66690704,  -22.87408242],
       ..., 
       [-192.58763491,  184.79552786, -163.98130449, ...,   -1.12378932,
          37.2343807 ,  -21.0776174 ],
       [-160.48657688,  151.70822638, -140.85556959, ...,   -1.75234939,
          35.67985213,  -15.21926118],
       [-162.64160268,  143.59671082, -105.44417801, ...,    0.28057274,
          31.427609  ,  -12.10403295]])

In [54]:
ts_labels[10]

array([ 1.,  0.])

In [9]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])


In [65]:
mfcc_size=13
num_labels=2
batch_size = 1128
hidden_size = 1024

graph = tf.Graph()
with graph.as_default():

    # Input data. For the training data, we use a placeholder that will be fed
    # at run time with a training minibatch.
    tf_train_dataset = tf.placeholder(tf.float32,shape=(batch_size, mfcc_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    #tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(ts_features)
    tf_test_dataset2 = tf.constant(single_features)
    # Variables.
    W1 = tf.Variable(tf.truncated_normal([mfcc_size, hidden_size]))
    b1 = tf.Variable(tf.zeros([hidden_size]))

    W2 = tf.Variable(tf.truncated_normal([hidden_size, num_labels]))
    b2 = tf.Variable(tf.zeros([num_labels]))
    #W1=tf.cast(W1, tf.float64)
    #b1=tf.cast(b1, tf.float64)
    #W2=tf.cast(W2, tf.float64)
    #b2=tf.cast(b2, tf.float64)
    # Training computation.
    y1 = tf.nn.relu( tf.matmul(tf_train_dataset, W1) + b1 )
    logits = tf.matmul(y1, W2) + b2
    
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=tf_train_labels))
    
    # Optimizer.
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
    
    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logits)

    #y1_valid = tf.nn.relu(tf.matmul(tf_valid_dataset, W1) + b1)
    #valid_logits = tf.matmul(y1_valid, W2) + b2
    #valid_prediction = tf.nn.softmax(valid_logits)
    w1=tf.cast(W1, tf.float64)
    B1=tf.cast(b1, tf.float64)
    w2=tf.cast(W2, tf.float64)
    B2=tf.cast(b2, tf.float64)
    y1_test = tf.nn.relu(tf.matmul(tf_test_dataset, w1) + B1)
    test_logits = tf.matmul(y1_test, w2) + B2
    test_prediction = tf.nn.softmax(test_logits)
    y2_test = tf.nn.relu(tf.matmul(tf_test_dataset2, w1) + B1)
    test_logits2 = tf.matmul(y2_test, w2) + B2
    test_prediction2 = tf.nn.softmax(test_logits2)
    print test_prediction2
    print test_prediction
    




Tensor("Softmax_2:0", shape=(10700, 2), dtype=float64)
Tensor("Softmax_1:0", shape=(175, 2), dtype=float64)


In [39]:
print graph

In [43]:
#sess = tf.InteractiveSession()
#a=tf.Print(test_prediction,[test_prediction])
#b = tf.add(a, a).eval(session=graph.as_default())

AttributeError: 'GeneratorContextManager' object has no attribute 'graph'

In [66]:
print y1_test

Tensor("Relu_1:0", shape=(175, 1024), dtype=float64)


In [67]:
# Let's run it:
num_steps = 3001

with tf.Session(graph=graph) as session:
    tf.initialize_all_variables().run()
    #saver = tf.train.Saver()
    print("Initialized")
    for step in range(num_steps):
        # Pick an offset within the training data, which has been randomized.
        # Note: we could use better randomization across epochs.
        offset = (step * batch_size) % (tr_labels.shape[0] - batch_size)
        # Generate a minibatch.
        batch_data = tr_features[offset:(offset + batch_size), :]
        batch_labels = tr_labels[offset:(offset + batch_size), :]
        #print batch_labels
        # Prepare a dictionary telling the session where to feed the minibatch.
        # The key of the dictionary is the placeholder node of the graph to be fed,
        # and the value is the numpy array to feed to it.
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 500 == 0):
            print np.shape(batch_data)
            print np.shape(predictions)           
            print np.shape(batch_labels)
            print("Minibatch loss at step %d: %f" % (step, l))
            print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
            #print("Validation accuracy: %.1f%%" % accuracy(valid_prediction.eval(), valid_labels))
            print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), ts_labels))
            print("Test accuracy 2: %.1f%%" % accuracy(test_prediction2.eval(), single_labels))
        #save_path = saver.save(session, "model.ckpt") #saves the cirrent model

    print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), ts_labels))
    print("Test accuracy 2: %.1f%%" % accuracy(test_prediction2.eval(), single_labels))    
    check2=test_prediction2.eval()
    print check2
    check1=test_prediction.eval()
    print check1

Instructions for updating:
Use `tf.global_variables_initializer` instead.
Initialized
(1128, 13)
(1128, 2)
(1128, 2)
Minibatch loss at step 0: 3099.464355
Minibatch accuracy: 70.9%
Test accuracy: 28.6%
Test accuracy 2: 50.0%
(1128, 13)
(1128, 2)
(1128, 2)
Minibatch loss at step 500: 0.416098
Minibatch accuracy: 86.7%
Test accuracy: 71.4%
Test accuracy 2: 50.0%
(1128, 13)
(1128, 2)
(1128, 2)
Minibatch loss at step 1000: 0.120458
Minibatch accuracy: 100.0%
Test accuracy: 71.4%
Test accuracy 2: 50.0%
(1128, 13)
(1128, 2)
(1128, 2)
Minibatch loss at step 1500: 0.141797
Minibatch accuracy: 100.0%
Test accuracy: 71.4%
Test accuracy 2: 50.0%
(1128, 13)
(1128, 2)
(1128, 2)
Minibatch loss at step 2000: 0.355552
Minibatch accuracy: 88.8%
Test accuracy: 71.4%
Test accuracy 2: 50.0%
(1128, 13)
(1128, 2)
(1128, 2)
Minibatch loss at step 2500: 0.118711
Minibatch accuracy: 100.0%
Test accuracy: 71.4%
Test accuracy 2: 50.0%
(1128, 13)
(1128, 2)
(1128, 2)
Minibatch loss at step 3000: 0.571963
Minibatch

In [68]:
check2

array([[ 0.87181889,  0.12818111],
       [ 0.87181889,  0.12818111],
       [ 0.87181889,  0.12818111],
       ..., 
       [ 0.87181889,  0.12818111],
       [ 0.87181889,  0.12818111],
       [ 0.87181889,  0.12818111]])

In [69]:
check1

array([[ 0.87181889,  0.12818111],
       [ 0.87181889,  0.12818111],
       [ 0.87181889,  0.12818111],
       [ 0.87181889,  0.12818111],
       [ 0.87181889,  0.12818111],
       [ 0.87181889,  0.12818111],
       [ 0.87181889,  0.12818111],
       [ 0.87181889,  0.12818111],
       [ 0.87181889,  0.12818111],
       [ 0.87181889,  0.12818111],
       [ 0.87181889,  0.12818111],
       [ 0.87181889,  0.12818111],
       [ 0.87181889,  0.12818111],
       [ 0.87181889,  0.12818111],
       [ 0.87181889,  0.12818111],
       [ 0.87181889,  0.12818111],
       [ 0.87181889,  0.12818111],
       [ 0.87181889,  0.12818111],
       [ 0.87181889,  0.12818111],
       [ 0.87181889,  0.12818111],
       [ 0.87181889,  0.12818111],
       [ 0.87181889,  0.12818111],
       [ 0.87181889,  0.12818111],
       [ 0.87181889,  0.12818111],
       [ 0.87181889,  0.12818111],
       [ 0.87181889,  0.12818111],
       [ 0.87181889,  0.12818111],
       [ 0.87181889,  0.12818111],
       [ 0.87181889,

In [19]:
#new model to be loaad by tf.restore
v1 = tf.Variable(11, name="v1")
v2 = tf.Variable(22, name="v2")

saver = tf.train.Saver()
with tf.Session() as sess:
    # Restore variables from disk.
    saver.restore(sess, "/tmp/model.ckpt")
    print("Model restored.")


NotFoundError: Unsuccessful TensorSliceReader constructor: Failed to find any matching files for /tmp/model.ckpt
	 [[Node: save/RestoreV2 = RestoreV2[dtypes=[DT_INT32], _device="/job:localhost/replica:0/task:0/cpu:0"](_recv_save/Const_0, save/RestoreV2/tensor_names, save/RestoreV2/shape_and_slices)]]
	 [[Node: save/Assign/_2 = _Recv[client_terminated=false, recv_device="/job:localhost/replica:0/task:0/gpu:0", send_device="/job:localhost/replica:0/task:0/cpu:0", send_device_incarnation=1, tensor_name="edge_12_save/Assign", tensor_type=DT_FLOAT, _device="/job:localhost/replica:0/task:0/gpu:0"]()]]

Caused by op u'save/RestoreV2', defined at:
  File "/usr/lib/python2.7/runpy.py", line 174, in _run_module_as_main
    "__main__", fname, loader, pkg_name)
  File "/usr/lib/python2.7/runpy.py", line 72, in _run_code
    exec code in run_globals
  File "/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py", line 3, in <module>
    app.launch_new_instance()
  File "/usr/local/lib/python2